In [82]:
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import copy
import math


path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "2016031115-LUN2.csv_updated_Heiner.csv"))

f = all_files[0]  # Change the file name as required
print("Working with file " + str(f))
cols = ['IO_num','LBA','Size','Deathtime_RWI']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
df['Deathtime_RWI'] = df['Deathtime_RWI'].replace(np.NaN, -1)
lba_list = df['LBA'].tolist()

print(len(lba_list))
print(len(df))

Working with file C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\2016031115-LUN2.csv_updated_Heiner.csv
273174
273174


In [83]:
TB = 1000000000000
GB = 1000000000
page_size = 4096
page_per_block = 10  # Hyper parameter
over_provisioning_ratio = 0.3
ssd_capacity = (0.5 * GB) 
GC_THRESHOLD = 3
P2L = {}
L2P = {}
closed_list_blocks = []
LOG_PAGE_PER_BLOCK = math.log(page_per_block,2)


# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter <= (ssd_capacity/page_size)):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)


print(len(page_addresses))
print(len(block_addresses))

122072
12208


In [84]:
# Defining block_structure
block_struct = {}
for x in free_list_block:
    block_start = x
    write_ptr=0
    valid_pages=0
    valid_bitmap = []
    for x in range(page_per_block):
        valid_bitmap.append(True)

    segment = [block_start,valid_pages,valid_bitmap,write_ptr]
    block_struct[block_start]=segment
    
print(len(block_struct))

12208


In [85]:
counter = 0
cur_block=free_list.pop()

while(counter < len(lba_list)):
    print(counter)
    lba=lba_list[counter]
    if lba in L2P:
        invalidate_lba(lba)
    map_lba(lba)
    check_GC(False)
    counter = counter + 1
    

0


TypeError: unsupported operand type(s) for <<: 'int' and 'float'

In [77]:
def invalidate_lba(lba):
    prev = L2P[lba]
    #note above how phys addresses are formed/concatenated
    prev_block = prev >> LOG_PAGE_PER_BLOCK
    prev_page = prev % PAGE_PER_BLOCK
    block_details = block_struct[prev_block]
    
    block_details[2] = block_details[2] - 1            # Decreasing valid pages
    block_details[3][prev_page] = False                # Setting bitmap to False
    
    
#map LBA to phys
def map_lba(lba):
    block_details = block_struct[cur_block]
    phys_addr = (block_details[0] << LOG_PAGE_PER_BLOCK)| block_details[3]
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba
    block_details[3] = block_details[3] + 1               # Setting valid pages
    block_details[2][block_details[3]] = True             # Setting Bitmap
    block_details[1] = block_details[1] + 1               # Increasing Write pointer


#check if we need to close/open block. Do not perform GC if we are already
#doing gc
def check_GC (gc_check):
    block_details = block_struct[cur_block]
    if (block_details[3] == PAGES_PER_BLOCK):
        closed_blocks.push(cur_block)
        #get new block
        if (len(free_list) < GC_THRESHOLD):
            if (gc_check != True):
                do_greedy_gc()
            else:
                if (len(free_list) == 0):
                    print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS")
                else:
                    #we are doing GC already, pick the next block from the free list
                    cur_block = free_list.pop()


def do_greedy_gc():
    for x in closed_blocks:
        min_val = float('inf')
        block_details = block_struct[x]                   
        if (block_details[1] < min_val):
            min_val = block_details[1]
        gc_blk = x
        #found the block with minimal valid pages, move all valid pages
        for pg in range(PAGES_PER_BLOCK):
            #figure out the logical addresses for all phys pages in the gc block
            phys_addr = (block_details[gc_blk][0] << LOG_PAGES_PER_BLOCK) | pg
            gc_lba = P2L[phys_addr]
            invalidate_lba(gc_lba)
            #move the gc'ed block to a new location
            map_lba(gc_lba)
            #check if we need to get a new block
            check_GC(True)